# Prelude

In [ ]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!pip install pytesseract
!pip install pdf2image
!pip install docx2txt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.86.1-0ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_sm

2023-05-03 21:33:13.884232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Data Preparation

The code below extracts from both word documets and pdf documents and returns a text of it for further analysis.

In [ ]:
import spacy
import docx2txt
import pandas as pd
import nltk

import spacy
import docx2txt
import pandas as pd
import nltk
from pdf2image import convert_from_path
import pytesseract

def extract_text_from_doc(doc_path):
    if doc_path.lower().endswith('.docx'):
        temp = docx2txt.process(doc_path)
    elif doc_path.lower().endswith('.pdf'):
        temp = extract_text_from_scanned_pdf(doc_path)
    else:
        raise ValueError("Unsupported file format. Please provide a PDF or DOCX file.")
    
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    return ' '.join(text)

def extract_text_from_scanned_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    temp = ''
    for img in images:
        img_text = pytesseract.image_to_string(img)
        temp += img_text + '\n'

    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    return ' '.join(text)

# Named Entity Recognition (NER)

From the extracted text we extract the name from each resume

In [ ]:
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # define the pattern
    pattern = [{'POS': 'PROPN', 'OP': '+'}]
    
    # initialize matcher with the pattern
    matcher = Matcher(nlp.vocab)
    matcher.add('NAME', [pattern])
    
    # get matches from the matcher
    matches = matcher(nlp_text)
    
    # iterate over the matches and extract the span
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        # Ensure that the matched tokens are consecutive proper nouns
        if all(token.pos_ == "PROPN" for token in span):
            return span.text


From the extracted text we extract the phone number from each resume.

In [ ]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

From the extracted text we use regex to extract the emails from each text.

In [ ]:
import re

def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

From the extracted text we extracted the skills.

In [ ]:
nltk.download('stopwords')
SKILLS_DB = [
'machine learning','data science','python','word','excel','java','javascript','sql','ruby','php','swift','objective-c','typescript','c++','go','rust','scala','haskell','html',
'css','react','angular','vue','node.js','express','django','flask','ruby','mysql','postgresql','oracle','mongodb','redis','cassandra',
'aws','azure','docker','scrum','git','funtional testing', 'algorithms and data structures','object-oriented programming','software architecture','rest apis'
]
 


def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)
 
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
 
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
 
    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
 
    # we create a set to keep the results in.
    found_skills = set()
 
    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)
 
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)
 
    return found_skills

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


From the extracted text we extracted education.

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
 
 
RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'Schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
]
 

 
 
def extract_education(input_text):
    organizations = []
 
    # first get all the organization names using nltk
    for sent in nltk.sent_tokenize(input_text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
                organizations.append(' '.join(c[0] for c in chunk.leaves()))
 
    # we search for each bigram and trigram for reserved words
    # (college, university etc...)
    education = set()
    for org in organizations:
        for word in RESERVED_WORDS:
            if org.lower().find(word) >= 0:
                education.add(org)
 
    return education

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


# Have a Try

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data  =extract_text_from_doc("/content/Rami-Lionel-Kuttab-Resume_SuggestedEdits.docx")
resume_name = "John Doe.docx"


In [ ]:
description = extract_text_from_doc("/content/About the job.docx")

In [ ]:
print(data)

Rami-Lionel Kuttab (917) 9945360 • rlk_23@outlook.com • linkedin.com/in/rami-kuttab/ EDUCATION Syracuse University, School of Engineering and Computer Science, Syracuse, NY   August 2022 – May 2024       M.S. in Computer Science Relevant Coursework: Operating Systems, Algorithms, Data Science, Computer Architecture, Structured Programming, Natural Language Processing Syracuse University, School of Engineering and Computer Science, Syracuse, NY                                      August 2018 – May 2022                                                                                                                B.S. in Computer Science Relevant Coursework: Data Structures, Algorithms, Agile Development, Database, Object Oriented in C++, Introduction into A.I., Operating Systems, Machine Learning, and Physics WORK EXPERIENCE Software Developer, BGC Partners, New York, New York                     June 2022 – August 2022 Developed and thoroughly tested the code, continuously refining and

In [ ]:
name =  extract_name(data)

In [ ]:
print(name)

Rami


In [ ]:
mobile = extract_mobile_number(data)

In [ ]:
print(mobile)

9179945360


In [ ]:
email = extract_email(data)

In [ ]:
print(email)

rlk_23@outlook.com


In [ ]:
skills = extract_skills(data)

In [ ]:
print(skills)

{'Java', 'SQL', 'Ruby', 'CSS', 'Word', 'Excel', 'Oracle', 'HTML', 'Data Science', 'PHP', 'React', 'Python', 'PostgreSQL', 'Machine Learning', 'JavaScript'}


In [ ]:
skills = list(skills)

In [ ]:
education = extract_education(data)

In [ ]:
print(education)

{'EDUCATION Syracuse University', 'School', 'Syracuse University'}


In [ ]:
import spacy

# load the English language model
nlp = spacy.load('en_core_web_sm')



# process the resume text with SpaCy
doc = nlp(data)

# iterate over named entities in the document
for ent in doc.ents:
    # print the entity text and its label
    print(ent.text, ent.label_)


Rami-Lionel Kuttab ORG
917 CARDINAL
University, School of Engineering and Computer Science ORG
Syracuse GPE
NY ORG
August 2022 DATE
May 2024 DATE
M.S. GPE
Algorithms PERSON
Structured Programming ORG
School of Engineering and Computer Science ORG
Syracuse GPE
August 2018 DATE
May 2022 DATE
B.S. GPE
Data Structures ORG
Algorithms PERSON
Introduction ORG
A.I. GPE
Operating Systems ORG
Machine Learning ORG
Physics ORG
BGC Partners ORG
New York GPE
New York GPE
June 2022 DATE
August 2022 DATE
25 % PERCENT
three CARDINAL
Pandas Library PERSON
12% PERCENT
July 2021 DATE
August 2021 DATE
WhatsApp ORG
React Native ORG
Utilized Firebase’s ORG
Front-End Developer                                                                   ORG
February 2021 – DATE
June 2021 DATE
HTML ORG
CSS ORG
JavaScript ORG
daily DATE
60 CARDINAL
HTML ORG
CSS ORG
Bootstrap Led PERSON
three CARDINAL
Spotify Recommendation System ORG
August 2022 DATE
December 2022 DATE
Used Collaborative Filtering WORK_OF_ART
Spotify FAC
S

# Matching


This code is an example of how to use natural language processing (NLP) techniques to match job skills in a job description to skills listed in a resume.

The code loads the English language model for NLP using the spacy library and defines the job description and resume as strings. It then creates a PhraseMatcher object and adds a list of skills to it. The code uses the matcher to extract relevant features from the job description and resume, converts the features into a feature matrix using a CountVectorizer, and calculates the cosine similarity between the feature vectors. Finally, the code prints the cosine similarity score.

Overall, this code can be used as a starting point for building an automated resume screening system that matches job skills to resumes, which can save time for HR professionals and recruiters.

In [ ]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from spacy.matcher import PhraseMatcher

# Load the English language model for NLP
nlp = spacy.load('en_core_web_sm')

# Define the resumes and job descriptions as strings
resume_text = data
job_text = description

# Define the list of skills to match against
#skills = ['Python', 'Java', 'software engineering']

# Create a PhraseMatcher object and add the skills to it
matcher = PhraseMatcher(nlp.vocab)
for skill in skills:
    matcher.add('Skill', None, nlp(skill))

# Convert the text into spaCy documents
resume_doc = nlp(resume_text)
job_doc = nlp(job_text)

# Extract the relevant features from the documents using the PhraseMatcher
resume_skills = [resume_doc[start:end].text for match_id, start, end in matcher(resume_doc)]
job_skills = [job_doc[start:end].text for match_id, start, end in matcher(job_doc)]

# Combine the skills into a single text string for vectorization
resume_skill_text = ' '.join(resume_skills)
job_skill_text = ' '.join(job_skills)

# Create a CountVectorizer to convert the skill text into a feature matrix
vectorizer = CountVectorizer()
try:
    skill_matrix = vectorizer.fit_transform([resume_skill_text, job_skill_text])
except ValueError:
    print('No valid features found in the input data')
    skill_matrix = None

if skill_matrix is not None:
    # Calculate the cosine similarity between the two feature vectors
    cosine_sim = cosine_similarity(skill_matrix[0], skill_matrix[1])

    # Print the cosine similarity score
    print('Cosine similarity: %.2f' % cosine_sim)
    result = cosine_sim


Cosine similarity: 0.27


In [ ]:
import csv

filename = 'Results.csv'

# Check if the file exists
try:
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        # If file already exists, we read the existing data
        data = list(reader)
except FileNotFoundError:
    # If file doesn't exist, create an empty list for data
    data = []

# Ask user for new data
new_resume = resume_name
if skill_matrix == None:
  new_value = "not relevant resume"
else:
  new_value = result[0][0]


# Check if the resume name already exists in the file
resume_names = [row[0] for row in data]
if new_resume in resume_names:
    print('Error: Resume name already exists in file')
else:
    # Add new data to the data list
    data.append([new_resume, new_value])

    # Write the data to the CSV file
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)


Error: Resume name already exists in file


In [ ]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from spacy.matcher import PhraseMatcher

def get_similarity(data,job_,skills):
  
  # Load the English language model for NLP
  nlp = spacy.load('en_core_web_sm')

  # Define the resumes and job descriptions as strings
  resume_text = data
  job_text = job_

  # Define the list of skills to match against
  #skills = ['Python', 'Java', 'software engineering']

  # Create a PhraseMatcher object and add the skills to it
  matcher = PhraseMatcher(nlp.vocab)
  for skill in skills:
      matcher.add('Skill', None, nlp(skill))

  # Convert the text into spaCy documents
  resume_doc = nlp(resume_text)
  job_doc = nlp(job_text)

  # Extract the relevant features from the documents using the PhraseMatcher
  resume_skills = [resume_doc[start:end].text for match_id, start, end in matcher(resume_doc)]
  job_skills = [job_doc[start:end].text for match_id, start, end in matcher(job_doc)]

  # Combine the skills into a single text string for vectorization
  resume_skill_text = ' '.join(resume_skills)
  job_skill_text = ' '.join(job_skills)

  # Create a CountVectorizer to convert the skill text into a feature matrix
  vectorizer = CountVectorizer()
  try:
      skill_matrix = vectorizer.fit_transform([resume_skill_text, job_skill_text])
  except ValueError:
      print('No valid features found in the input data')
      skill_matrix = None

  if skill_matrix is not None:
      # Calculate the cosine similarity between the two feature vectors
      cosine_sim = cosine_similarity(skill_matrix[0], skill_matrix[1])

      # Print the cosine similarity score
      print('Cosine similarity: %.2f' % cosine_sim)
      result = cosine_sim
  return result[0][0]

In [ ]:
final_result = []

In [ ]:
from google.colab import drive
drive.mount('/drive')
data = extract_text_from_doc("/content/Rami-Lionel-Kuttab-Resume_SuggestedEdits.docx")
print(data)
name =  extract_name(data)
print(name)
mobile = extract_mobile_number(data)
print(mobile)
email = extract_email(data)
print(email)
skills = extract_skills(data)
print(skills)
education = extract_education(data)
print(education)
result = get_similarity(data,description,skills)
print(get_similarity(data,description,skills))
final_result.append(("Rami-Lionel-Kuttab",result))



Mounted at /drive
Rami-Lionel Kuttab (917) 9945360 • rlk_23@outlook.com • linkedin.com/in/rami-kuttab/ EDUCATION Syracuse University, School of Engineering and Computer Science, Syracuse, NY   August 2022 – May 2024       M.S. in Computer Science Relevant Coursework: Operating Systems, Algorithms, Data Science, Computer Architecture, Structured Programming, Natural Language Processing Syracuse University, School of Engineering and Computer Science, Syracuse, NY                                      August 2018 – May 2022                                                                                                                B.S. in Computer Science Relevant Coursework: Data Structures, Algorithms, Agile Development, Database, Object Oriented in C++, Introduction into A.I., Operating Systems, Machine Learning, and Physics WORK EXPERIENCE Software Developer, BGC Partners, New York, New York                     June 2022 – August 2022 Developed and thoroughly tested the code, continu

# Further Analysis

In [ ]:

data = extract_text_from_doc("/content/CCC resume.docx")
print(data)
name =  extract_name(data)
print(name)
mobile = extract_mobile_number(data)
print(mobile)
email = extract_email(data)
print(email)
skills = extract_skills(data)
print(skills)
education = extract_education(data)
print(education)
result = get_similarity(data,description,skills)
print(get_similarity(data,description,skills))
final_result.append(("CCC",result))

Chih-Chia, Chen cchen64@syr.edu ▪ Syracuse, New York ▪ (315)-777-3497 linkedin.com/in/ChihChia-Chen/  SUMMARY:    I’m looking for internship and full-time role opportunities in software engineering and security.   EDUCATION:  Syracuse University, College of Engineering and Computer Science Fall 2022  Bachelor’s in computer science  Syracuse University, College of Engineering and Computer Science Fall 2024  Master’s in computer science  SKILLS:  Programming: Python, JAVA, C/C++, SQL, Data Structures, Software Specification & Design Applications: Microsoft Visual Studio Code, Microsoft Visual Studio, Microsoft SQL Server Management Studio Languages: English (Bilingual), Mandarin (Bilingual), Japanese (elementary)    RELATED COURSEWORK: Object Oriented Programming C++ Software Specification & Design Database Design SQL Software Security Basic Hardware/software Security Data Structures (Java) Object Oriented Programming Python Algorithms Network and Web Security Agile Software Development 

In [ ]:

data = extract_text_from_scanned_pdf("/content/Sai sreeramNachireddi_resume.pdf")
print(data)
name =  extract_name(data)
print(name)
mobile = extract_mobile_number(data)
print(mobile)
email = extract_email(data)
print(email)
skills = extract_skills(data)
print(skills)
education = extract_education(data)
print(education)
result = get_similarity(data,description,skills)
print(get_similarity(data,description,skills))
final_result.append(("Sai",result))

Sai Sreeram Nachireddi 321 Westcott Street, Apt. 4, Syracuse, NY, 13210 | 680-697-4695 | ssrn.2021@gmail.com | https:/Mwww.linkedin.com/in/nachireddi-sai-sreeram-16414023a   EDUCATION Syracuse University - College of Engineering & Computer Science, Syracuse, NY August 2022 - May 2024 Master of Science in Computer Science Courses: Principles of Operating Systems, Design & Analysis of Algorithms, Natural Language Processing GPA: 3.44/4 Jawaharlal Nehru Technological University - Gokaraju Rangaraju Institute of Engineering & Technology, Hyderabad, India June 2018 - June 2022 Bachelor of Technology in Computer Science & Engineering Courses: Data Structures, Algorithms, Artificial Intelligence, Database Management Systems, Web Development GPA: 8.25/10 PROFESSIONAL EXPERIENCE Programmer Analyst Intern, Cognizant — Hyderabad, India January 2022 - June 2022 e Developed a search filtering feature using Java programming language and by deploying Spring Boot e Managed and collaborated closely wit

In [ ]:
data = extract_text_from_scanned_pdf("/content/Resume-UD.pdf")
print(data)
name =  extract_name(data)
print(name)
mobile = extract_mobile_number(data)
print(mobile)
email = extract_email(data)
print(email)
skills = extract_skills(data)
print(skills)
education = extract_education(data)
print(education)
result = get_similarity(data,description,skills)
print(get_similarity(data,description,skills))
final_result.append(("UD",result))

Uday Kumar Putta 966 Lancaster Ave, Syracuse,NY | +1315 664 6584 | uputta@syr.edu EDUCATION Syracuse University - College of Engineering & Computer Science, Syracuse, NY August 2022 - May 2024 Master of Science in Cyber Security * Course Work: Cryptography, Design Analysis and Algorithm, Operating Systems Gandhi Institute of Technology and Management - School of technology, Hyderabad, July 2018 - May 2022 Telangana Bachelor of Technology , Computer Science, June 2022 * CGPA: 8.03/10 * Data Mining and Data Warehousing, Formal Languages and Automata Theory, Software Engineering , Cloud computing, Machine Learning, Cryptography and Network Security. EXPERIENCE cyber Security Intern, VERZEO, Hyderabad, Telangana June 2021 - September 2021 Performed Foot printing on Amazon website and registrant information IP Address using shodan, Netcraft, DNS dumpster. + Implemented SQL Injections on websites to access the data through credentials. * Utilized set tool in kali linux created fake gmail to 

In [ ]:
data = extract_text_from_scanned_pdf("/content/Rajashekar_Resume.pdf")
print(data)
name =  extract_name(data)
print(name)
mobile = extract_mobile_number(data)
print(mobile)
email = extract_email(data)
print(email)
skills = extract_skills(data)
print(skills)
education = extract_education(data)
print(education)
print(get_similarity(data,description,skills))

RAJASHEKAR REDDY MORA (315) 454-1924 * ramora@syr.edu * www.linkedin.com/in/rajashekar-reddy-mora   EDUCATION   Syracuse University, School of Information Studies, Syracuse, NY May 2024 M.S. Applied Data Science Relevant Coursework: Natural Language Processing | Data Administration and Data management | Introduction to Data Science | Database Management | Communication for Information Management Professionals National Institute of Technology | Calicut, India May 2019 B.Tech. Electrical and Electronics Engineering Relevant Coursework: Data Structures and Algorithms, Computer programming, Image Processing WORK EXPERIENCE   Applications Systems Coordinator, M.H.Alshaya Co. , Bangalore, India July 2019 — Jun 2022 e Extracted and analyzed retail data sources using SQL and Python to aid the Brand Marketing team with insights to drive informed decision making e Working on a sentiment analysis project aimed at automatically gauging the nature of the feedback provided to the IT help desk. Emplo

In [ ]:
data = extract_text_from_doc("/content/John Doe.docx")
print(data)
name =  extract_name(data)
print(name)
mobile = extract_mobile_number(data)
print(mobile)
email = extract_email(data)
print(email)
skills = extract_skills(data)
print(skills)
education = extract_education(data)
print(education)
result = get_similarity(data,description,skills)
print(get_similarity(data,description,skills))
final_result.append(("JohnDoe",result))

John Doe 123 Main St. | Anytown, USA 12345 | john.doe@email.com | (123) 456-7890 Objective: Software engineer with over 5 years of experience seeking a challenging position in a dynamic company that values innovation and excellence. Experience: Software Engineer ABC Company | Anytown, USA | 2018 - Present - Designed and developed custom software solutions for clients using Python and Java - Worked collaboratively with project managers and clients to define project requirements and deliverables - Mentored junior engineers and provided technical leadership to project teams Software Engineer XYZ Company | Anytown, USA | 2015 - 2018 - Designed and developed software applications for internal use using C++ and Python - Collaborated with cross-functional teams to ensure software met business requirements - Implemented unit and integration testing to ensure high-quality software Education: Bachelor of Science in Computer Science Anytown University | Anytown, USA | 2015 Skills: - Proficient in

In [ ]:
print(final_result)

[('Rami-Lionel-Kuttab', 0.2672612419124244), ('CCC', 0.6396021490668313), ('Sai', 0.5), ('UD', 0.35355339059327373), ('JohnDoe', 0.5345224838248488)]


In [ ]:
with open('Final_result.csv', mode="w", newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Resume-Name', 'Similarity'])
    for row in final_result:
        writer.writerow(row)
        print(file.closed) # Check if file is still open
    

False
False
False
False
False
